<a href="https://colab.research.google.com/github/Efradgalio/Employee-Sentiment-Tracker/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [14]:
# !pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [16]:
# !pip install accelerate[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# Sentiment Analysist Use Pretrained Model BERT Uncased + Fine tuned  


In [3]:
df = pd.read_excel('/content/drive/MyDrive/AI FOR INDONESIA/Final Project/lowercase_labels_datasets.xlsx')

## pre processing:
- drop N/A

In [4]:
df.head()

,text,sentiment
0,deserved candidates are promoted promptly.\nun...,1
1,you got lot of learning platform and monthly l...,1
2,based on which business unit you are you will ...,1
3,some client projects are good as they use the ...,1
4,"i have worked in ibm (4 years) ,tcs (1 year) ,...",1


In [5]:
# Remove rows with missing or invalid text data
df.dropna(subset=['text', 'sentiment'], inplace=True)

## preparation for sentiment analysis, consists:
- encode labels
- split data
- tokenize the text column on train and val

In [9]:
# Encode sentiment labels
label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [10]:
# Split data into train and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2)


In [11]:
# Debugging: Print first few entries
print("First 5 train texts:", train_texts[:5])
print("First 5 train labels:", train_labels[:5])
print("First 5 val texts:", val_texts[:5])
print("First 5 val labels:", val_labels[:5])

First 5 train texts: ['as of now nothing from my end because everything going smooth for me.', 'nothing to like here', 'connect with other team to due team work', 'everything', 'just the compensation part']
First 5 train labels: [0, 1, 1, 1, 0]
First 5 val texts: ['bad behavior in bench time, work life balance is total on client wish for good or bad environment, appraisal is poor and in some project senior are doing micro management.', 'work atmosphere', 'now a days the way of handling projects become too difficult.', 'the best environment to work and very understanding manager . work flexibility is there and many more.', 'best work culture']
First 5 val labels: [0, 1, 0, 1, 1]


In [12]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 classes: negative and positive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Tokenize data
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=128)
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=128)


In [14]:
# Create torch dataset
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [15]:
!pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


## fine tuned, results, and save.

The fine tuned consists:
- epoch: 3
-  bath size: 16

In [16]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [17]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [18]:
# Train model
trainer.train()

Step,Training Loss
10,0.722500
20,0.736300
30,0.718100
40,0.703800
50,0.714700
60,0.689500
70,0.687900
80,0.666800
90,0.668100
100,0.651800


TrainOutput(global_step=7032, training_loss=0.5734468095080845, metrics={'train_runtime': 2592.8517, 'train_samples_per_second': 43.38, 'train_steps_per_second': 2.712, 'total_flos': 7398617098959360.0, 'train_loss': 0.5734468095080845, 'epoch': 3.0})

In [19]:
# Evaluate model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6024132966995239, 'eval_runtime': 66.7705, 'eval_samples_per_second': 140.391, 'eval_steps_per_second': 8.776, 'epoch': 3.0}


In [35]:
#import matplotlib.pyplot as plt

# Get the training logs
# train_logs = trainer.callback_handler.log_history

# Extract the training loss from the logs
# train_loss = [log["loss"] for log in train_logs if "loss" in log]

# Plot the training loss
# plt.plot(train_loss, label="Training Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("Training Loss Over Epochs")
# plt.legend()
# plt.show()

AttributeError: 'CallbackHandler' object has no attribute 'log_history'

In [20]:
# Save model
model.save_pretrained("/content/drive/MyDrive/AI FOR INDONESIA/Final Project/01_09062024_bert_sentiment_model")

## testing the fine tuned models!

In [36]:
# Load tokenizer and fine-tuned model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/AI FOR INDONESIA/Final Project/01_09062024_bert_sentiment_model") # the fine tuned model save

# Function to tokenize text and obtain BERT embeddings
def tokenize_and_predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits
    return predictions

# Function to compute percentage
def compute_percentage(predictions):
    softmax_output = torch.softmax(predictions, dim=1)
    positive_percentage = softmax_output[:, 1].item() * 100
    negative_percentage = 100 - positive_percentage
    return positive_percentage, negative_percentage

In [44]:
# User input
user_text = input("")

# Tokenize user input and get predictions
user_predictions = tokenize_and_predict(user_text)
positive_percentage, negative_percentage = compute_percentage(user_predictions)

# Determine sentiment
sentiment = "Positive" if user_predictions.argmax() == 1 else "Negative"

print("Sentiment:", sentiment)
print("Positive sentiment percentage:", positive_percentage)
print("Negative sentiment percentage:", negative_percentage)

Very low salary, poor working conditions, very little training provided but high expectations.
Sentiment: Negative
Positive sentiment percentage: 12.681084871292114
Negative sentiment percentage: 87.31891512870789


# Sentimen Analysist Use pretrained Model

In [45]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm



In [46]:
# Load dataset
data = pd.read_excel('/content/drive/MyDrive/AI FOR INDONESIA/Final Project/lowercase_text_only_datasets.xlsx')



In [47]:
data.head()

,text
0,deserved candidates are promoted promptly.\nun...
1,you got lot of learning platform and monthly l...
2,based on which business unit you are you will ...
3,some client projects are good as they use the ...
4,"i have worked in ibm (4 years) ,tcs (1 year) ,..."


In [48]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")



In [49]:
# Function to tokenize text and obtain BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # Average pooling over token embeddings



In [ ]:
# Apply BERT analysis to each text in the dataset
output_embeddings = []
for text in tqdm(data['text'].astype(str)):
    embeddings = get_bert_embeddings(text)
    output_embeddings.append(embeddings)



 92%|█████████▏| 49949/54026 [1:34:57<06:43, 10.10it/s]

In [1]:
# Add the BERT analysis results to the dataset as a new column
data['output_Bert'] = output_embeddings

# Save the updated dataset
data.to_csv('output_dataset.csv', index=False)

NameError: name 'output_embeddings' is not defined